# Mediods Analysis

This experiment has the purpose of analysing the medoid of some of the clusters defined in the previous experiment. According to it, the best number of cluster is 32 according with elbow method.

In [1]:
#!pip install numpy==1.15 wordcloud

In [2]:
import sys
import os
sys.path.append('../')
from src import reader as r
from src import visualization as v

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
%matplotlib notebook
import numpy as np
import pandas as pd
import sklearn
print(sklearn.__version__)
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances

0.20.0


In [4]:
X = r.readWord2Vec(normed=False)
npX = X.values
print(X[:10])
print(X.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
        0         1         2         3         4         5         6    \
0  0.013595 -0.068772 -0.013126  0.006024  0.037922 -0.020872 -0.010325   
1  0.025009 -0.025586  0.012237  0.020567  0.014523 -0.018705 -0.058395   
2  0.031544 -0.025882 -0.060617  0.035992  0.037742 -0.021132 -0.041023   
3  0.058714 -0.080251 -0.026101  0.060461  0.016714 -0.005205 -0.037441   
4  0.023366 -0.065667 -0.015601  0.022602  0.030365 -0.060590 -0.071055   
5  0.005894 -0.074649 -0.001171  0.014182  0.029530  0.025231 -0.022154   
6 -0.011415 -0.021273 -0.043368 -0.001785 -0.031960 -0.020326 -0.071195   
7  0.049307 -0.024540 -0.011390  0.010187  0.017417 -0.000969 -0.020045   
8  0.052583 -0.090634 -0.025581  0.003195  0.031459  0.002013  0.025611   
9  0.061589 -0.026826 -0.020978  0.046825  0.002022 -0.016367 -0.058929   

        7         8         9      ...          118       119       120  \
0  0.008019  0.0

In [5]:
from importlib import reload
reload(r)
news = r.readNews()
print(news.head(10))
print(news.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
Read 13229 lines
                   id                    publish_date  \
0  576880531301801984  Sat Mar 14 23:00:11 +0000 2015   
1  576820122666471424  Sat Mar 14 19:00:08 +0000 2015   
2  576744652717461504  Sat Mar 14 14:00:15 +0000 2015   
3  576736754436304896  Sat Mar 14 13:28:52 +0000 2015   
4  576736614766010368  Sat Mar 14 13:28:18 +0000 2015   
5  576548368740052992  Sat Mar 14 01:00:17 +0000 2015   
6  576518190286536704  Fri Mar 13 23:00:22 +0000 2015   
7  576494177971732480  Fri Mar 13 21:24:57 +0000 2015   
8  576472874946433024  Fri Mar 13 20:00:18 +0000 2015   
9  576464606551490560  Fri Mar 13 19:27:27 +0000 2015   

                                       headline_test  
0  An abundance of online info can turn us into e...  
1  A plant-based diet that incorporates fish may ...  
2  It doesn't take much to damage your hearing at...  
3  RT @CNN: Forever young? Discover this island’s...

In [6]:
from MulticoreTSNE import MulticoreTSNE as TSNE

smp_sz = X.size
tsne_bow = TSNE(n_components=2, perplexity=10, verbose=True, n_jobs=-1)#500
tsne_bow_result = tsne_bow.fit_transform(X)

## Computing the Medoids
In here, we compute the medoids as the closest point to the centroids of the clusters.

In [7]:
best_K = 22
print("#############################")
print("Best K =", best_K)
print("Applying K-means")
best_cluster = KMeans(n_clusters=best_K, n_jobs=-1)
best_cluster_result = best_cluster.fit(X)
print("Finished")
print("#############################")    

#############################
Best K = 22
Applying K-means
Finished
#############################


In [8]:
y_pred = best_cluster_result.labels_
arrays=[]
for i in range(best_K):
    dist = np.linalg.norm(best_cluster_result.cluster_centers_[i]-npX[y_pred==i],axis=1)
    index = np.argmin(dist)
    arrays.append(npX[y_pred==i][index])   

medoids = np.vstack(arrays)
print(medoids.shape)
print(medoids)

(22, 128)
[[ 0.03621743 -0.02559954  0.0001907  ...  0.009918   -0.026162
  -0.04250124]
 [ 0.02881752 -0.00065937 -0.02970518 ...  0.03434596 -0.04455674
  -0.02181085]
 [ 0.04371578 -0.00270727 -0.01084024 ...  0.02332169 -0.03504989
  -0.00562588]
 ...
 [ 0.05481039 -0.04737444  0.0006567  ...  0.05074434  0.01596553
  -0.03167734]
 [ 0.0326329  -0.0393059   0.03538762 ...  0.01153963 -0.05382344
   0.02522494]
 [ 0.05162259 -0.09289704  0.01216706 ...  0.04618941 -0.01589652
  -0.04103516]]


In [9]:
l = []
for i in range(best_K):
    l.append(np.where(np.all(npX==medoids[i],axis=1))[0][0]) # these last [0][0] return the index, since the output of this command is (array([id]),)
    
medoids_index = np.array(l)
print(medoids_index)
print(len(np.unique(medoids_index)))

[ 9292 10105  2040  1810  7972  1672  1999  6268   567  7194  3079  7725
   414  8131  1469  1115  6536  7898   554  3626  7717  6889]
22


## Selecting medoids neighbors
Here the medoids neighbors are selected in order to plot them and see if the clusters make sense.

In [10]:
nm = 5 #number of medoids to be analyzed
nn = 5 #number of neighbor of each medoids
np.random.seed(42)
selected_medoids_index = np.random.randint(0,best_K,nm)
selected_medoids = medoids[selected_medoids_index]
print("Random indexes:",selected_medoids_index)
indexes_per_medoid = []

for i in range(nm):
    p = selected_medoids[i]
    aux = npX[y_pred==selected_medoids_index[i]]
    d = np.linalg.norm(aux-p,axis=1)
    points = []
    points.append(medoids_index[selected_medoids_index[i]])
    
    for j in range(nn):
        min_d = d.argmin()
        while (d[min_d] == 0.):
            d[min_d] = np.inf
            min_d = d.argmin()
        points = points + (np.where(np.all(npX==aux[min_d],axis=1))[0]).tolist()
        d[min_d] = np.inf
        
    indexes_per_medoid.append(np.unique(points).tolist())

    
print("Points:",indexes_per_medoid)
print(len(indexes_per_medoid))

Random indexes: [ 6 19 14 10  7]
Points: [[322, 1999, 2096, 2599, 5801, 10501], [1380, 1666, 1837, 3626, 7341, 8235], [1468, 1469, 3400, 9658, 9753, 10465], [1571, 2120, 3079, 3177, 4796, 7987], [303, 1333, 5454, 5566, 6268, 8841]]
5


## Analyzing the Medoids
Here we start the analysis of the medoids and their neighbors.

### Exporting the news of the medoids and clusters

In [11]:
import os 

os.makedirs('../output/w2vec_clusters/', exist_ok=True)
os.makedirs('../output/w2vec_medoids/', exist_ok=True)

print("Random indexes:",selected_medoids_index)
for i in range(best_K):
    np.savetxt("../output/w2vec_clusters/k_"+str(i)+".txt", news[y_pred==i].values, fmt='%s', encoding='utf8')
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    np.savetxt("../output/w2vec_medoids/k_"+str(selected_medoids_index[i])+".txt", np.take(news,p,0).values, fmt='%s')

Random indexes: [ 6 19 14 10  7]


### News of the medoids and their neighbors

In [12]:
for p in indexes_per_medoid:
    info = np.take(news,p,0).loc[:,"headline_test"].values
    print("----------------------------")
    for i in range(info.size):
        print(info[i],"\n")
print("----------------------------")

----------------------------
American #Ebola patient Dr. Rick Sacra has been released from the hospital 

Health Hero of the Day! Hospital staff sews up kid's favorite stuffed animal during surgery 

Kentucky hospital settles lawsuit over pediatric program 

Who knew? Copper in hospital rooms may stop infections 

In mysterious case, hospital sees second death from cyanide poisoning 

VIDEO: PM challenged on girl's hospital letter 

----------------------------
RT @denisermt: @cnnhealth A4) For free time playtime, we visit the park, playground or the beach for unstructured, let kids be kids playtim… 

RT @NAMICommunicate: @EAVrotsos Funding for #mentalhealth varies by state, yes. For the latest snapshot on $ in states, 

Testosterone replacement is the hot new therapy for men. But "T" could be dangerous for your hubby's heart 

RT @UNAIDS reports a gap of US$7.2 bn in 2011 to reach the global #HIV targets for low &amp; middle income countries for 2015 

RT @EinsteinMed: A7: It has been

### Generating cloud of words

In [13]:
%matplotlib inline
from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt

os.makedirs('../output/w2vec_wordcloud_clusters/', exist_ok=True)
os.makedirs('../output/w2vec_wordcloud_medoids/', exist_ok=True)

# clusters
for i in range(best_K):
    info = ' '.join(news[y_pred==i].loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/w2vec_wordcloud_clusters/k_"+str(i))
    plt.close(fig)

#medoids
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    info = ' '.join(np.take(news,p,0).loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/w2vec_wordcloud_medoids/k_"+str(selected_medoids_index[i]))
    plt.close(fig)
